In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rand

from sklearn.preprocessing import MinMaxScaler

In [2]:
import os
os.chdir('C:\\code\\activ')

In [3]:
df_name = 'nov_nine_var.xlsx'

In [4]:
df = pd.read_excel('./documents/'+df_name).iloc[:,1::]
X = df.iloc[:,0:21]
y = df.iloc[:,21:22].to_numpy()

In [5]:
y[-1]

array([0.50384917])

In [6]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [7]:
dnn_model = tf.keras.models.load_model('./model/dnn.h5')

In [12]:
def my_loss(real_data, gen_data):
    dist = tf.math.sqrt(tf.reduce_sum(tf.math.square(tf.math.subtract(real_data, gen_data))))
    y_pred = dnn_model(gen_data)

    a = tf.cast(tf.math.subtract(y_pred, 0.50384917), dtype=tf.float32)
    b = tf.cast(dist, dtype=tf.float32)

    loss = tf.math.add(a, b)
    return loss

In [13]:
input_layer = tf.keras.layers.Input(shape=(21,))

encoded = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(64, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(32, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(16, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(3)(encoded)
 
decoded = tf.keras.layers.Dense(3, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(16, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(32, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(64, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(128, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(21, activation='sigmoid')(decoded)
 
# autoencoder
autoencoder = tf.keras.Model(input_layer, decoded)
 
# encoder
encoder = tf.keras.Model(input_layer, encoded)

# decoder
encoded_input = tf.keras.layers.Input(shape=(3,))

decoded_layer = autoencoder.layers[6](encoded_input)
decoded_layer = autoencoder.layers[7](decoded_layer)
decoded_layer = autoencoder.layers[8](decoded_layer)
decoded_layer = autoencoder.layers[9](decoded_layer)
decoded_layer = autoencoder.layers[10](decoded_layer)
decoded_layer = autoencoder.layers[11](decoded_layer)

decoder = tf.keras.Model(encoded_input, decoded_layer)

In [14]:
autoencoder.compile(optimizer='adam', loss=my_loss, metrics=['mse'])

In [15]:
result = []
pop_result = []
for i in range(1000):
    X_train = X[rand.randint(0, 394)].reshape(-1, 1).T
    X_pred = autoencoder.predict(X_train, verbose=0)

    loss = my_loss(X_train, X_pred)
    autoencoder.train_on_batch(X_pred, X_train)

    pop = dnn_model.predict(X_pred, verbose=0)
    
    result.append(X_pred)
    pop_result.append(pop)

    print(f"epoch: {i}, loss: {loss}, pop: {pop}")

epoch: 0, loss: [[1.4579744]], pop: [[0.47508758]]
epoch: 1, loss: [[0.98028827]], pop: [[0.4749005]]
epoch: 2, loss: [[1.0747935]], pop: [[0.4748323]]
epoch: 3, loss: [[1.1901782]], pop: [[0.47490877]]
epoch: 4, loss: [[0.8990194]], pop: [[0.47477943]]
epoch: 5, loss: [[0.9714005]], pop: [[0.47460377]]
epoch: 6, loss: [[1.4720404]], pop: [[0.474795]]
epoch: 7, loss: [[1.580221]], pop: [[0.47477165]]
epoch: 8, loss: [[1.3750329]], pop: [[0.47449502]]
epoch: 9, loss: [[1.4718188]], pop: [[0.47449622]]
epoch: 10, loss: [[1.3388711]], pop: [[0.47434017]]
epoch: 11, loss: [[1.0957778]], pop: [[0.4742382]]
epoch: 12, loss: [[1.473331]], pop: [[0.4740505]]
epoch: 13, loss: [[1.3331118]], pop: [[0.47378093]]
epoch: 14, loss: [[1.3987634]], pop: [[0.47386965]]
epoch: 15, loss: [[1.5875143]], pop: [[0.47353482]]
epoch: 16, loss: [[0.89722157]], pop: [[0.4728831]]
epoch: 17, loss: [[0.9120474]], pop: [[0.47293168]]
epoch: 18, loss: [[1.180736]], pop: [[0.47243056]]
epoch: 19, loss: [[0.9869197]]

In [16]:
gen = autoencoder.predict(X)

14/14 [==============================] - 0s 2ms/step


In [17]:
print(dnn_model.predict(gen, verbose=0).max())
print(dnn_model.predict(gen, verbose=0).min())
print(dnn_model.predict(gen, verbose=0).mean())

0.4944428
0.43980992
0.45840335


In [18]:
latest = np.array([X[-1]])
latest_gen = autoencoder.predict(latest)
dist = np.sqrt(np.sum(np.square(latest-latest_gen)))

print(dist)

1/1 [==============================] - 0s 35ms/step
2.6359505750674255


In [19]:
result = np.array(result).reshape(1000, 21)
X_result = scaler.inverse_transform(result)
pop_result = np.array(pop_result).reshape(1000, 1)

In [20]:
df = np.append(X_result, pop_result, axis=1)

In [21]:
pd_df = pd.DataFrame(df)
pd_df.to_excel('./documents/result/ae_formula.xlsx')